In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.pipeline import Pipeline
from functions import *
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [45]:
dates = pd.read_csv('df_dates_test.csv')
metrics = pd.read_csv('df_metrics_test.csv')
opens = pd.read_csv('df_opens_test.csv')
pred = pd.read_csv('df_pred_test.csv')
original = pd.read_csv('etfs_dataset_modelling.csv')

In [46]:
dates.rename(columns={'Value':'Dates_test'}, inplace=True)
dates.drop('Unnamed: 0', axis=1, inplace=True)

opens.rename(columns={'Value':'Open_test'}, inplace=True)
opens.drop(['Ticker', 'Unnamed: 0'], axis=1, inplace=True)

pred.rename(columns={'Value':'Close_pred_test'}, inplace=True)
pred.drop(['Ticker', 'Unnamed: 0'], axis=1, inplace=True)


original.drop(['Unnamed: 0'], axis=1, inplace=True)


In [47]:
df

,Open_test,Ticker,Dates_test,Close_pred_test
0,45.520000,AADR,2023-03-24,45.759169
1,46.040001,AADR,2023-03-27,46.681240
2,47.049999,AADR,2023-03-28,46.993237
3,47.410000,AADR,2023-03-29,47.445742
4,48.209999,AADR,2023-03-30,48.081131
...,...,...,...,...
382301,19.500000,ZHDG,2024-07-02,19.320602
382302,19.673000,ZHDG,2024-07-03,19.496974
382303,19.768999,ZHDG,2024-07-05,19.686856
382304,19.889999,ZHDG,2024-07-08,19.689868


In [48]:
original['Dates_test'] = pd.to_datetime(original['Date'])
df['Dates_test'] = pd.to_datetime(df['Dates_test'])

In [51]:
merged = original.merge(df, on=['Dates_test', 'Ticker'])
merged.columns

Index(['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'RSI_9', 'RSI_14', 'RSI_25', 'ewma_7', 'ewma_50',
       'ewma_200', 'Close_target', 'Dates_test', 'Open_test',
       'Close_pred_test'],
      dtype='object')

In [53]:
merged.drop(['Date', 'Open', 'High', 'Low','Volume', 'Dividends',
       'Stock Splits', 'RSI_9', 'RSI_14', 'RSI_25', 'ewma_7', 'ewma_50',
       'ewma_200', 'Close_target'], axis=1, inplace=True)

In [55]:
merged.to_csv('df_final.csv')

In [56]:
merged

,Ticker,Close,Dates_test,Open_test,Close_pred_test
0,BNE,19.710978,2023-10-25,19.710978,20.003800
1,BNE,19.603056,2023-10-26,19.683254,19.998725
2,BNE,19.392164,2023-10-27,19.392164,19.996904
3,BNE,19.457510,2023-10-30,19.359489,19.996904
4,BNE,19.510975,2023-10-31,19.346619,19.996904
...,...,...,...,...,...
382301,VRP,23.940001,2024-07-02,23.760000,23.926052
382302,VRP,24.040001,2024-07-03,23.910000,23.989946
382303,VRP,24.049999,2024-07-05,24.049999,24.021057
382304,VRP,24.070000,2024-07-08,24.070000,24.028760


In [9]:
df = pd.concat([opens, dates, pred], axis=1)
df

,Open_test,Ticker,Dates_test,Close_pred_test
0,45.520000,AADR,2023-03-24,45.759169
1,46.040001,AADR,2023-03-27,46.681240
2,47.049999,AADR,2023-03-28,46.993237
3,47.410000,AADR,2023-03-29,47.445742
4,48.209999,AADR,2023-03-30,48.081131
...,...,...,...,...
382301,19.500000,ZHDG,2024-07-02,19.320602
382302,19.673000,ZHDG,2024-07-03,19.496974
382303,19.768999,ZHDG,2024-07-05,19.686856
382304,19.889999,ZHDG,2024-07-08,19.689868


In [14]:
df_final = df

df_final.to_csv('df_final.csv')

In [30]:
tickers = df.groupby('Ticker').count()
print(tickers)
print(tickers.max())
print(tickers.min())

        Open_test  Dates_test  Close_pred_test
Ticker                                        
AADR          324         324              324
AAPB           91          91               91
AAPD           91          91               91
AAPY           31          31               31
AAXJ          324         324              324
...           ...         ...              ...
YANG          324         324              324
YOLO          259         259              259
YXI           324         324              324
YYY           324         324              324
ZHDG          147         147              147

[1485 rows x 3 columns]
Open_test          324
Dates_test         324
Close_pred_test    324
dtype: int64
Open_test          18
Dates_test         18
Close_pred_test    18
dtype: int64


In [31]:
df['Dates_test'] = pd.to_datetime(df['Dates_test'])

# Ordenar datos por Ticker y fecha
df = df.sort_values(by=['Ticker', 'Dates_test'])

df['Open_15d'] = df.groupby('Ticker')['Open_test'].shift(15).dropna()

df.dropna(inplace=True)

# Calcular las rentabilidades diarias
df['Rentabilidad_diaria'] = (df['Close_pred_test'] - df['Open_15d']) / df['Open_15d']

df_sorted = df.sort_values(by='Dates_test', ascending=False)
df_sorted

,Open_test,Ticker,Dates_test,Close_pred_test,Open_15d,Rentabilidad_diaria
382305,19.799999,ZHDG,2024-07-09,19.638636,19.440001,0.010218
321915,47.615002,SKOR,2024-07-09,47.635395,47.492085,0.003018
355666,45.137001,UTWY,2024-07-09,45.225888,45.729732,-0.011018
72769,31.184999,ECML,2024-07-09,31.578939,31.506001,0.002315
346829,21.530001,TUA,2024-07-09,21.508895,21.459628,0.002296
...,...,...,...,...,...,...
260520,63.869999,PBE,2023-04-17,64.078698,60.189999,0.064607
143221,110.250000,FXH,2023-04-17,110.319128,102.980003,0.071267
324534,54.549999,SMMD,2023-04-17,54.828612,51.680000,0.060925
143545,29.389999,FXI,2023-04-17,29.616670,28.620001,0.034824


In [32]:
ultima_fecha = df.groupby('Ticker')['Dates_test'].max().reset_index()
ultima_fecha

# Paso 2: Merge para obtener los datos de la última fecha
df_ultima_fecha = pd.merge(df, ultima_fecha, on=['Ticker', 'Dates_test'])
df_ultima_fecha.to_csv('etfs_ultima_fecha_15d.csv')

In [33]:
df_ultima_fecha

,Open_test,Ticker,Dates_test,Close_pred_test,Open_15d,Rentabilidad_diaria
0,62.860001,AADR,2024-07-09,62.992689,62.630001,0.005791
1,29.610001,AAPB,2024-07-09,28.857473,26.240000,0.099751
2,16.860001,AAPD,2024-07-09,19.411267,17.943572,0.081795
3,27.500000,AAPY,2024-07-09,27.133608,27.000000,0.004948
4,73.849998,AAXJ,2024-07-09,74.063437,71.669998,0.033395
...,...,...,...,...,...,...
1480,8.530000,YANG,2024-07-09,8.655882,8.600000,0.006498
1481,3.260000,YOLO,2024-07-09,3.154076,3.500000,-0.098835
1482,17.219999,YXI,2024-07-09,17.213201,17.299999,-0.005017
1483,12.010000,YYY,2024-07-09,12.098302,11.930100,0.014099


In [49]:
from scipy.optimize import minimize

# Definir el capital total
capital_total = 3500 

# Crear una función objetivo para maximizar el rendimiento total
def objetivo(pesos, rendimientos, capital_total):
    return -np.sum(pesos * rendimientos)  # Negativo porque queremos maximizar el rendimiento

# Crear una función de restricción que la suma de los pesos sea igual a 1
def restriccion(pesos):
    return np.sum(pesos) - 1

def restriccion_capital(pesos):
    return np.sum(pesos * capital_total) - capital_total

top_10_etfs = df_ultima_fecha.nlargest(10, 'Rentabilidad_diaria')

# Datos para la optimización
rendimientos = top_10_etfs['Rentabilidad_diaria'].values
num_etfs = len(top_10_etfs)
pesos_iniciales = np.ones(num_etfs) / num_etfs  # Distribución inicial equitativa

# Definir las restricciones y los límites
restricciones = [{'type': 'eq', 'fun': restriccion},
                 {'type': 'ineq', 'fun': restriccion_capital}]

limites = [(0, 1)] * num_etfs  # Los pesos deben estar entre 0 y 1

# Ejecutar la optimización
resultado = minimize(objetivo, pesos_iniciales, args=(rendimientos, capital_total), method='SLSQP', bounds=limites, constraints=restricciones)

# Obtener los pesos óptimos
pesos_optimos = resultado.x

top_10_etfs['peso'] = pesos_optimos
top_10_etfs['inversion'] = top_10_etfs['peso'] * capital_total

# Convertir pesos a porcentajes de inversión
top_10_etfs['porcentaje_inversion'] = (top_10_etfs['peso'] * 100).apply(lambda x: "{:,.2f}%".format(x))

# Aplicar formato a la columna 'inversion'
top_10_etfs['Inversion_$'] = top_10_etfs['inversion'].apply(lambda x: "{:,.2f}".format(x))



In [50]:
top_10_etfs

,Open_test,Ticker,Dates_test,Close_pred_test,Open_15d,Rentabilidad_diaria,peso,inversion,porcentaje_inversion,Inversion_$
1341,14.510000,TSLL,2024-07-09,15.760786,8.275000,0.904627,2.582720e-01,9.039520e+02,25.83%,903.95
158,24.370001,CARU,2024-07-09,23.988226,17.799999,0.347653,4.733333e-01,1.656666e+03,47.33%,"1,656.67"
1438,19.020000,WEBL,2024-07-09,20.050642,16.000000,0.253165,2.077669e-01,7.271842e+02,20.78%,727.18
113,31.549999,BNKU,2024-07-09,32.878812,27.510000,0.195159,4.487558e-02,1.570645e+02,4.49%,157.06
1342,17.059999,TSLY,2024-07-09,17.189498,14.508583,0.184781,1.575223e-02,5.513279e+01,1.58%,55.13
1340,27.768000,TSLH,2024-07-09,27.735458,23.575001,0.176478,0.000000e+00,0.000000e+00,0.00%,0.00
880,9.220000,KORU,2024-07-09,9.177515,7.910000,0.160242,6.931955e-17,2.426184e-13,0.00%,0.00
1352,38.590000,UCC,2024-07-09,39.039052,34.200001,0.141493,8.318346e-17,2.911421e-13,0.00%,0.00
1351,22.330000,UBR,2024-07-09,22.983087,20.660000,0.112444,1.732989e-16,6.065461e-13,0.00%,0.00
571,29.636000,GDMN,2024-07-09,30.071038,27.139000,0.108038,0.000000e+00,0.000000e+00,0.00%,0.00


In [51]:
top_10_etfs.to_csv('top_10_etfs.csv')